In [1]:
import sys
sys.path.append('..\..')

In [174]:
import pandas as pd
import numpy as np
from config import *
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statistics as stat
pd.set_option('display.max_columns', 500)
from functools import reduce
from math import sqrt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn import tree
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


In [72]:
df = pd.read_csv(path_arquivo_treino)
df = df.drop(df[df.vh_weight < 50].index)
df.shape

(217636, 26)

In [151]:
feat = pd.read_csv('pred_file.csv')

In [124]:
df = df[df['year']<4]
df.shape

(163227, 26)

In [152]:
feat

,id_policy,year,pred
0,PL000000,4,0
1,PL042495,4,0
2,PL042496,4,0
3,PL042497,4,0
4,PL042498,4,0
...,...,...,...
54404,PL002373,4,1
54405,PL004062,4,0
54406,PL006847,4,0
54407,PL012984,4,0


In [75]:
Xraw = df.drop(columns=['claim_amount'])
yraw = df['claim_amount'].values

In [159]:
def concat_by_id(df,pred):
    dfs=[]
    lista_info=[]
    df_ano_1 = df[df['year']==1]
    if int(max(df['year']))==1:
        df_merged = df
    else:
        for i in range(1,int(max(df['year']))): #add anos 1-max
            df_aux = df[df['year']==i]
            df_aux = df_aux.rename(columns={'claim_amount':'claim_amount_'+str(i),'pol_no_claims_discount':'pol_no_claims_discount_'+str(i)})
            dfs.append(df_aux[['id_policy','claim_amount_'+str(i),'pol_no_claims_discount_'+str(i)]])
            lista_info = lista_info + ['claim_amount_'+str(i),'pol_no_claims_discount_'+str(i)]
        df_ano_pred = df[df['year']==max(df['year'])] #add ano de predição
        df_ano_pred = df_ano_pred.rename(columns={'claim_amount':'claim_amount_'+str(int(max(df['year'])))})
        dfs.append(df_ano_pred[['id_policy','claim_amount_'+str(int(max(df['year'])))]])
        
        df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['id_policy'],
                                            how='inner'), dfs)
        
        dfs = [df_ano_1[['id_policy', 'pol_coverage',
       'pol_duration', 'pol_sit_duration', 'pol_pay_freq', 'pol_payd',
       'pol_usage', 'drv_sex1', 'drv_age1', 'drv_age_lic1', 'drv_drv2',
       'drv_sex2', 'drv_age2', 'drv_age_lic2', 'vh_make_model', 'vh_age',
       'vh_fuel', 'vh_type', 'vh_speed', 'vh_value', 'vh_weight', 'population',
       'town_surface_area']],df_merged,feat[['id_policy','pred']]]
        df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['id_policy'],
                                            how='inner'), dfs)
    return df_merged,lista_info

def carros_prob(df):
    carros_maior_prob = ['aewtdnpoiopumymt','asmpttrlkodaejic','dgwbxitzfzbegnoc',
                         'dlrodwgixwmoquny','mdxtphkujabwpjeu','oryfrzxilushvigq',
                         'qxtqrwxfvuenelml','sguprofjftozaujc','synvsxhrexuyxpre',
                         'wehkqzwvbeonajcu','wxzfbqtarfurwcfw','yvlkrzgjhwrlyihc']
    carros_menor_prob = ['cgkclpnidlmetsrb','dllcylnkzeegtsgr','dwsasdexwmpsmowl',
                         'epbwnmcyogpybxlm','hpohizpkyzvwunni','hwldevoubgzgbhgs',
                         'jgkpiuuctpywtrlh','ldxjynecsqlswvbq','nhwgapjtnadqqaul',
                         'pjbnwqhnqczouirt','prtnwsypyfnshpqx','shemwbbeliuvnvvm',
                         'wyqgeeclrqbihfpk','zxvcbwcwoqnkxxbs']
    return carros_maior_prob,carros_menor_prob

def preprocessing(Xraw,yraw,pred):
    df_total = Xraw
    df_total['claim_amount'] = yraw #gerando o df original
    carros_maior_prob,carros_menor_prob = carros_prob(df_total) #gerando a lista de carros
    df_total = df_total.drop(df_total[df_total.vh_weight < 50].index) # drop de dados sem sentido
    df_merged_by_id,lista_info = concat_by_id(df_total,pred) #df id por linha
    
    df_merged_by_id['pol_coverage_2'] = df_merged_by_id['pol_coverage'].map({'Min':1,'Med1':2,'Med2':3,'Max':4})
    
    df_fuel = pd.get_dummies(df_merged_by_id['vh_fuel'], drop_first = True, prefix = 'fuel')
    df_merged_by_id = pd.concat([df_merged_by_id,df_fuel], axis=1)
    
    df_merged_by_id['grupo_risco_1'] = ((df_merged_by_id['pol_usage']=='Professional')&
                                        (df_merged_by_id['pol_coverage']=='Max')).astype(int)
    
    df_merged_by_id['grupo_risco_2'] = ((df_merged_by_id['pol_usage']=='WorkPrivate')&
                                        (df_merged_by_id['pol_coverage']=='Min')).astype(int)
    
    df_merged_by_id['grupo_risco_3'] = df_merged_by_id['vh_make_model'].isin(carros_maior_prob).astype(int)
    
    df_merged_by_id['grupo_risco_4'] = df_merged_by_id['vh_make_model'].isin(carros_menor_prob).astype(int)
    
    df_merged_by_id['grupo_risco_5'] = ((df_merged_by_id['vh_age']>15) ).astype(int)
    
    df_merged_by_id['grupo_risco_6'] = ((df_merged_by_id['vh_age']>0)&
                                        (df_merged_by_id['vh_age']<6)).astype(int)
    
    df_merged_by_id['vh_value_risk'] = df_merged_by_id['vh_value']/df_merged_by_id['pol_coverage_2']
    
    features = ['pol_coverage_2','drv_age_lic1', 'vh_weight',
                'fuel_Gasoline','drv_age1','vh_speed','vh_age',
                'vh_value','population','grupo_risco_1',
                'grupo_risco_2','grupo_risco_3','grupo_risco_4',
                'grupo_risco_5','grupo_risco_6','vh_value_risk','pred']
    
    df_aux = df_merged_by_id.rename(columns = {'claim_amount_'+str(int(max(df_merged_by_id['year']))):'y'})
    Xraw = df_aux[features+lista_info].copy()
    yraw = df_aux['y']
    Xraw.fillna(0,inplace = True)
    
    return Xraw,yraw


In [160]:
X,y = preprocessing(Xraw,yraw,feat)

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#### Regressão Linear

In [187]:
reg = LinearRegression().fit(X_train, y_train)

In [188]:
reg.score(X_train,y_train)

0.02261507864562795

In [189]:
reg.score(X_test,y_test)

0.022336163730348524

In [193]:
y_pred = reg.predict(X)
sqrt(mean_squared_error(y,y_pred))

543.8782534004982

In [199]:
final = y_pred-y

In [204]:
final[final<0]

22      -3526.852377
24      -1000.153189
54         -8.401128
86      -2821.619932
104      -919.034396
            ...     
54354   -2035.343384
54374   -1091.243082
54383    -333.590734
54384     -85.075873
54385   -1079.646417
Name: y, Length: 5923, dtype: float64

#### Decision Tree

In [272]:
clf = tree.DecisionTreeRegressor(max_depth=5,random_state=0)

In [273]:
clf = clf.fit(X_train, y_train)

In [274]:
clf.score(X_test,y_test)

0.01839630421088312

In [275]:
y_pred = clf.predict(X)
sqrt(mean_squared_error(y,y_pred))

538.6443372893756

In [276]:
final = y_pred-y

In [277]:
final[final<0]

22      -3588.017675
24       -971.436004
35        -27.867675
86      -2604.106231
104      -954.536004
            ...     
54354   -2016.931635
54374   -1075.486004
54383    -417.107675
54384    -126.263921
54385   -1113.546004
Name: y, Length: 4986, dtype: float64